<a href="https://colab.research.google.com/github/AdityaDhiman05/UPI-Fraud-Detection-Project/blob/main/DAY_3_CAPSTONE_PROJECT_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 3: Data Cleaning & Preprocessing 🔧

Goal:
- Clean raw UPI monthly data
- Fix types & missing values
- Engineer basic features (Year, Month, rolling averages, pct change)
- Save cleaned CSV & visuals for Day 3


Loading data and inspecting it

In [2]:
from google.colab import files
import pandas as pd

# Upload CSV
uploaded = files.upload()

# Load into dataframe (replace filename if different)
df = pd.read_csv("upi_data_enhanced.csv")
df.head()


Saving upi_data_enhanced.csv to upi_data_enhanced.csv


,Month,No. of Banks live on UPI,Volume (in Mn),Value (in Cr.),Avg_Txn_Value_INR,MoM_Growth_Volume_%,MoM_Growth_Value_%
0,Aug-25,688,20008.31,2485472.91,1242.220312,2.775639,-0.917887
1,Jul-25,684,19467.95,2508498.09,1288.527087,5.832777,4.349851
2,Jun-25,675,18395.01,2403930.69,1306.838480,-1.512251,-4.389550
3,May-25,673,18677.46,2514297.01,1346.166454,4.381722,4.984335
4,Apr-25,668,17893.42,2394925.87,1338.439421,-2.229816,-3.322098


Will inspect dataset now

In [4]:
print('shape:',df.shape)
print('columns:',df.columns.tolist())
df.info()
print('Are there any missing values:',df.isnull().sum())

shape: (113, 7)
columns: ['Month', 'No. of Banks live on UPI', 'Volume (in Mn)', 'Value (in Cr.)', 'Avg_Txn_Value_INR', 'MoM_Growth_Volume_%', 'MoM_Growth_Value_%']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Month                     113 non-null    object 
 1   No. of Banks live on UPI  113 non-null    int64  
 2   Volume (in Mn)            113 non-null    float64
 3   Value (in Cr.)            113 non-null    float64
 4   Avg_Txn_Value_INR         110 non-null    float64
 5   MoM_Growth_Volume_%       110 non-null    float64
 6   MoM_Growth_Value_%        110 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 6.3+ KB
Are there any missing values: Month                       0
No. of Banks live on UPI    0
Volume (in Mn)              0
Value (in Cr.)              0
Avg_Txn_Value_INR       

Renaming columns for easy recognition

In [6]:
df=df.rename(columns={
             "No. of Banks live on UPI":"Banks Live",
             "Volume (in Mn)":"Volume_Mn",
             "Value (in CR.)":"Value_Cr"})
df.head()

,Month,Banks Live,Volume_Mn,Value (in Cr.),Avg_Txn_Value_INR,MoM_Growth_Volume_%,MoM_Growth_Value_%
0,Aug-25,688,20008.31,2485472.91,1242.220312,2.775639,-0.917887
1,Jul-25,684,19467.95,2508498.09,1288.527087,5.832777,4.349851
2,Jun-25,675,18395.01,2403930.69,1306.838480,-1.512251,-4.389550
3,May-25,673,18677.46,2514297.01,1346.166454,4.381722,4.984335
4,Apr-25,668,17893.42,2394925.87,1338.439421,-2.229816,-3.322098


Converting Month to datetime

In [12]:
# Reload the dataset fresh without the failed conversion
df = pd.read_csv("upi_data_enhanced.csv")


print(df['Month'].head(20))
df['Month'] = pd.to_datetime(df['Month'], format='%b-%y', errors='coerce')
df = df.sort_values('Month').reset_index(drop=True)

df[['Month']].head(12)




0     Aug-25
1     Jul-25
2     Jun-25
3     May-25
4     Apr-25
5     Mar-25
6     Feb-25
7     Jan-25
8     Dec-24
9     Nov-24
10    Oct-24
11    Sep-24
12    Aug-24
13    Jul-24
14    Jun-24
15    May-24
16    Apr-24
17    Mar-24
18    Feb-24
19    Jan-24
Name: Month, dtype: object


,Month
0,2016-04-01
1,2016-05-01
2,2016-06-01
3,2016-07-01
4,2016-08-01
5,2016-09-01
6,2016-10-01
7,2016-11-01
8,2016-12-01
9,2017-01-01


In [14]:
for col in ['Volume (in Mn)', 'Value (in Cr.)']:
    print(col, df[col].head())
df[col] = pd.to_numeric(df[col].astype(str).str.replace(",",""), errors='coerce')


Volume (in Mn) 0    0.00
1    0.00
2    0.00
3    0.09
4    0.09
Name: Volume (in Mn), dtype: float64
Value (in Cr.) 0    0.00
1    0.00
2    0.00
3    0.38
4    3.09
Name: Value (in Cr.), dtype: float64


In [15]:
df = df.interpolate(method='linear')
print(df.isnull().sum())


Month                       0
No. of Banks live on UPI    0
Volume (in Mn)              0
Value (in Cr.)              0
Avg_Txn_Value_INR           3
MoM_Growth_Volume_%         3
MoM_Growth_Value_%          3
dtype: int64


## Feature Engineering: Extracting Date Components 🗓️

The goal of this step is to create new columns from the `Month` column to make analysis easier:

- **Year** → Extracts the year from the month (e.g., 2025). Useful for yearly trend analysis.
- **Month_Num** → Converts the month to a numeric value (1 = Jan, 12 = Dec). Useful for plotting and modeling. As numeric values are prefered over text values.
- **Quarter** → Determines the financial quarter (Q1–Q4). Helps to analyze seasonal trends.

These new features will help us later in visualizations and detecting trends over time.




In [16]:
df['Year'] = df['Month'].dt.year
df['Month_Num'] = df['Month'].dt.month
df['Quarter'] = df['Month'].dt.quarter

Will do a quick data check after all correction

In [17]:
df.head(10)
df.describe()

,Month,No. of Banks live on UPI,Volume (in Mn),Value (in Cr.),Avg_Txn_Value_INR,MoM_Growth_Volume_%,MoM_Growth_Value_%,Year,Month_Num,Quarter
count,113,113.000000,113.000000,1.130000e+02,110.000000,110.000000,110.000000,113.000000,113.000000,113.000000
mean,2020-11-30 12:19:06.902654976,272.539823,5153.367522,7.781111e+05,1786.563888,inf,inf,2020.460177,6.477876,2.495575
min,2016-04-01 00:00:00,21.000000,0.000000,0.000000e+00,42.222222,-19.831735,-26.795036,2016.000000,1.000000,1.000000
25%,2018-08-01 00:00:00,114.000000,312.020000,5.421226e+04,1479.764205,0.292026,0.424712,2018.000000,4.000000,2.000000
50%,2020-12-01 00:00:00,207.000000,2234.160000,4.161762e+05,1656.446707,6.678305,6.172378,2020.000000,6.000000,2.000000
75%,2023-04-01 00:00:00,414.000000,8863.260000,1.415505e+06,1833.802222,12.758849,17.757848,2023.000000,9.000000,3.000000
max,2025-08-01 00:00:00,688.000000,20008.310000,2.514297e+06,4857.000000,inf,inf,2025.000000,12.000000,4.000000
std,NaN,208.902533,5964.246855,8.287286e+05,703.857367,NaN,NaN,2.751738,3.404559,1.102950


In [20]:
df.columns.tolist()
df = df.rename(columns={
    "No. of Banks live on UPI": "Banks_Live",
    "Volume (in Mn)": "Volume_Mn",
    "Value (in Cr.)": "Value_Cr"
})
df.head(10)


,Month,Banks_Live,Volume_Mn,Value_Cr,Avg_Txn_Value_INR,MoM_Growth_Volume_%,MoM_Growth_Value_%,Year,Month_Num,Quarter
0,2016-04-01,21,0.00,0.00,NaN,NaN,NaN,2016,4,2
1,2016-05-01,21,0.00,0.00,NaN,NaN,NaN,2016,5,2
2,2016-06-01,21,0.00,0.00,NaN,NaN,NaN,2016,6,2
3,2016-07-01,21,0.09,0.38,42.222222,inf,inf,2016,7,3
4,2016-08-01,21,0.09,3.09,343.333333,0.000000,713.157895,2016,8,3
5,2016-09-01,25,0.09,32.64,3626.666667,0.000000,956.310680,2016,9,3
6,2016-10-01,26,0.10,48.57,4857.000000,11.111111,48.805147,2016,10,4
7,2016-11-01,30,0.29,100.46,3464.137931,190.000000,106.835495,2016,11,4
8,2016-12-01,35,1.99,707.93,3557.437186,586.206897,604.688433,2016,12,4
9,2017-01-01,36,4.46,1696.22,3803.183857,124.120603,139.602786,2017,1,1


Saving new dataset

In [21]:
df.to_csv("upi_cleaned.csv", index=False)

from google.colab import files
files.download("upi_cleaned.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>